In [1]:
import requests
import pandas as pd
from bs4 import BeautifulSoup
import time
import json
import altair as alt
import re
from sklearn.preprocessing import MinMaxScaler

In [2]:
game_details = pd.read_csv("steam_game_details.csv")
game_stats = pd.read_csv("steam_game_stats.csv")
game_reviews = pd.read_csv("steam_review_data.csv")

game_reviews = game_reviews.dropna()

Want to see whether positive ratio and number of owners is correlated to see how much positive reviews impact a game's performance.

In [4]:
combined_details = game_details.merge(game_stats, on="app_id")
combined_details['owners_upper'] = combined_details['owners'].apply(
    lambda x: int(re.search(r"\.\. ([\d,]+)", x).group(1).replace(',', '')) if pd.notnull(x) else None
)

combined_details['total'] = combined_details['positive'] + combined_details['negative']

combined_details['positive_ratio'] = combined_details['positive'] / combined_details['total']

In [5]:
temp = combined_details
temp['tags'] = temp['tags'].astype(str)
chart = alt.Chart(temp).mark_circle(size=60).encode(
    x=alt.X('positive_ratio', title='Positive Review Ratio'),
    y=alt.Y('owners_upper', title='Upper Bound of Owners'),
    tooltip=['title', 'positive_ratio', 'owners_upper', 'tags'],
).properties(
    width=600,
    height=400,
    title='Steam Game Popularity Scatterplot'
)

trend_line = alt.Chart(temp).transform_regression(
    'positive_ratio', 'owners_upper'
).mark_line(color='red').encode(
    x='positive_ratio',
    y='owners_upper'
)
chart + trend_line

/opt/anaconda3/lib/python3.12/site-packages/altair/utils/core.py:395: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)
/opt/anaconda3/lib/python3.12/site-packages/altair/utils/core.py:395: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)
/opt/anaconda3/lib/python3.12/site-packages/altair/utils/core.py:395: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)
/opt/anaconda3/lib/python3.12/site-packages/altair/utils/c

alt.LayerChart(...)

To create a metric for rating a game's popularity, rank them to [write why you change to rank them]. it is used to minimize outliers

In [6]:
scaler = MinMaxScaler()

combined_details['ranked_positive'] = combined_details['positive_ratio'].rank(pct=True)
combined_details['ranked_owners'] = combined_details['owners_upper'].rank(pct=True)

combined_details['popularity_score_ranked'] = (
    0.3 * combined_details['ranked_positive'] +
    0.7 * combined_details['ranked_owners']
)

combined_details['popularity_rating'] = (combined_details['popularity_score_ranked'] * 5).round(1)


In [7]:
histogram = alt.Chart(combined_details).mark_bar().encode(
    x = alt.X("popularity_rating"),
    y = alt.Y("count()")
)
histogram

/opt/anaconda3/lib/python3.12/site-packages/altair/utils/core.py:395: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)
/opt/anaconda3/lib/python3.12/site-packages/altair/utils/core.py:395: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)
/opt/anaconda3/lib/python3.12/site-packages/altair/utils/core.py:395: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)
/opt/anaconda3/lib/python3.12/site-packages/altair/utils/c

alt.Chart(...)

In [8]:
combined_details

,app_id,title,genre,developer,publisher,franchise,release_date,positive,negative,owners,price,tags,owners_upper,total,positive_ratio,ranked_positive,ranked_owners,popularity_score_ranked,popularity_rating
0,10,Counter-Strike,Action,Valve,Valve,NaN,"Nov 1, 2000",242768,6388,"10,000,000 .. 20,000,000",999,"{'Action': 5499, 'FPS': 4924, 'Multiplayer': 3...",20000000,249156,0.974361,0.994663,0.991667,0.992566,5.0
1,1002,Rag Doll Kung Fu,Indie,Mark Healey,Mark Healey,NaN,"Oct 12, 2005",89,30,"20,000 .. 50,000",99,"{'2D Fighter': 47, 'Martial Arts': 41, 'Intent...",50000,119,0.747899,0.406271,0.146667,0.224548,1.1
2,100400,Silo 2,Animation & Modeling,Nevercenter Ltd. Co.,Nevercenter Ltd. Co.,NaN,"Dec 19, 2012",61,24,"0 .. 20,000",5999,"{'Animation & Modeling': 23, 'Software': 11}",20000,85,0.717647,0.346898,0.042667,0.133936,0.7
3,10090,Call of Duty: World at War,Action,Treyarch,Activision,Call of Duty,"Nov 18, 2008",46341,3870,"2,000,000 .. 5,000,000",1499,"{'Zombies': 1197, 'World War II': 902, 'FPS': ...",5000000,50211,0.922925,0.875917,0.938333,0.919609,4.6
4,100980,3DCoat 4.9,"Animation & Modeling, Design & Illustration, E...",Pilgway,Pilgway,NaN,"Oct 1, 2012",129,17,"0 .. 20,000",0,"{'Animation & Modeling': 38, 'Utilities': 22, ...",20000,146,0.883562,0.751167,0.042667,0.255217,1.3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1495,259490,Beast Boxing Turbo,"Action, Indie, Sports",NaN,NaN,NaN,NaN,263,31,"0 .. 20,000",0,"{'Sports': 48, 'Action': 39, 'Boxing': 37, 'In...",20000,294,0.894558,0.795864,0.042667,0.268626,1.3
1496,259510,Shufflepuck Cantina Deluxe,"Action, Indie, Sports",Agharta Studio,Agharta Studio,NaN,"Dec 6, 2013",451,82,"100,000 .. 200,000",999,"{'Sports': 397, 'Indie': 60, 'VR': 52, 'Action...",200000,533,0.846154,0.633089,0.409000,0.476227,2.4
1497,259530,Savant - Ascent,"Action, Indie",D-Pad Studio,NaN,Savant - Ascent,"Dec 4, 2013",2583,219,"200,000 .. 500,000",199,"{'Action': 125, 'Indie': 111, 'Great Soundtrac...",500000,2802,0.921842,0.869246,0.595000,0.677274,3.4
1498,259550,Hero of the Kingdom,"Adventure, Casual, Indie, RPG",Lonely Troops,Lonely Troops,Hero of the Kingdom,"Dec 20, 2012",4376,296,"200,000 .. 500,000",799,"{'Casual': 182, 'Adventure': 133, 'Indie': 108...",500000,4672,0.936644,0.917945,0.595000,0.691884,3.5


Exploding genre so that i can make a vis of genre vs popularity rating

In [ ]:
genre_df = combined_details.dropna(subset='genre').copy()

# handles different results on Mac and Windows
if isinstance(genre_df['genre'].iloc[0], list):
    genre_df['genre'] = genre_df['genre'].apply(lambda lst: [g.strip() for g in lst])
else:
    genre_df['genre'] = genre_df['genre'].apply(lambda x: [g.strip() for g in x.split(',')])

exploded_genre_df = genre_df.explode('genre')

In [51]:
grouped_genre = exploded_genre_df.groupby('genre')['popularity_rating'].mean().reset_index()

alt.Chart(grouped_genre).mark_bar().encode(
    x = alt.X('genre:N', sort='-y'),
    y = 'popularity_rating'
)

/opt/anaconda3/lib/python3.12/site-packages/altair/utils/core.py:395: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)


alt.Chart(...)

In [70]:
publisher_count = combined_details.groupby("publisher").size().reset_index(name="count")

top_publishers = publisher_count[publisher_count['count'] >= 10]
top_merged_publishers = top_publishers.merge(combined_details, on="publisher").groupby("publisher")['popularity_rating'].mean().reset_index()
top_merged_publishers = top_merged_publishers.sort_values(by="popularity_rating", ascending=False)

# bottom_publishers = publisher_count[publisher_count['count'] < 10]
# bottom_merged_publishers = bottom_publishers.merge(combined_details, on="publisher").groupby("publisher")['popularity_rating'].mean().reset_index()
# bottom_merged_publishers = bottom_merged_publishers.sort_values(by="popularity_rating", ascending=False)

top = alt.Chart(top_merged_publishers).mark_bar().encode(
    x = alt.X('publisher:N', sort='-y'),
    y = 'popularity_rating'
)

# bottom = alt.Chart(bottom_merged_publishers).mark_bar().encode(
#     x = alt.X('publisher:N', sort='-y'),
#     y = 'popularity_rating'
# )
top

/opt/anaconda3/lib/python3.12/site-packages/altair/utils/core.py:395: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)


alt.Chart(...)

In [64]:
import datetime

format_string = "%b %d, %Y"

dates = combined_details.dropna(subset='release_date')
dates['release_date'] = dates['release_date'].apply(lambda x: datetime.datetime.strptime(x, format_string))

dates['release_year'] = dates['release_date'].apply(lambda x: x.year)
dates = dates.groupby('release_year')['popularity_rating'].mean().reset_index()

alt.Chart(dates).mark_bar().encode(
    x = alt.X('release_year:N', sort='-y'),
    y = 'popularity_rating'
)

/var/folders/z3/f13cd9js5xsdr5tmg66mr3f40000gn/T/ipykernel_19948/275777805.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dates['release_date'] = dates['release_date'].apply(lambda x: datetime.datetime.strptime(x, format_string))
/var/folders/z3/f13cd9js5xsdr5tmg66mr3f40000gn/T/ipykernel_19948/275777805.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dates['release_year'] = dates['release_date'].apply(lambda x: x.year)


alt.Chart(...)

FEATURE ENGINEERING FOR BUILDING PREDICTIVE MODEL

In [72]:
one_hot_publisher = pd.get_dummies(combined_details[['publisher']], prefix=['publisher'])

feature_df = pd.concat([combined_details, one_hot_publisher], axis=1).drop('publisher', axis=1)

one_hot_genre = pd.get_dummies(exploded_genre_df['genre'], prefix='genre')
exploded_encoded = pd.concat([exploded_genre_df, one_hot_genre], axis=1)
genre_encoded = exploded_encoded.groupby(level=0).max()

feature_df = feature_df.drop('genre', axis=1).join(genre_encoded.filter(like='genre_'))

feature_df

,app_id,title,developer,franchise,release_date,positive,negative,owners,price,tags,...,genre_Photo Editing,genre_RPG,genre_Racing,genre_Simulation,genre_Software Training,genre_Sports,genre_Strategy,genre_Utilities,genre_Video Production,genre_Web Publishing
0,10,Counter-Strike,Valve,NaN,"Nov 1, 2000",242768,6388,"10,000,000 .. 20,000,000",999,"{'Action': 5499, 'FPS': 4924, 'Multiplayer': 3...",...,False,False,False,False,False,False,False,False,False,False
1,1002,Rag Doll Kung Fu,Mark Healey,NaN,"Oct 12, 2005",89,30,"20,000 .. 50,000",99,"{'2D Fighter': 47, 'Martial Arts': 41, 'Intent...",...,False,False,False,False,False,False,False,False,False,False
2,100400,Silo 2,Nevercenter Ltd. Co.,NaN,"Dec 19, 2012",61,24,"0 .. 20,000",5999,"{'Animation & Modeling': 23, 'Software': 11}",...,False,False,False,False,False,False,False,False,False,False
3,10090,Call of Duty: World at War,Treyarch,Call of Duty,"Nov 18, 2008",46341,3870,"2,000,000 .. 5,000,000",1499,"{'Zombies': 1197, 'World War II': 902, 'FPS': ...",...,False,False,False,False,False,False,False,False,False,False
4,100980,3DCoat 4.9,Pilgway,NaN,"Oct 1, 2012",129,17,"0 .. 20,000",0,"{'Animation & Modeling': 38, 'Utilities': 22, ...",...,True,False,False,False,False,False,False,True,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1495,259490,Beast Boxing Turbo,NaN,NaN,NaN,263,31,"0 .. 20,000",0,"{'Sports': 48, 'Action': 39, 'Boxing': 37, 'In...",...,False,False,False,False,False,True,False,False,False,False
1496,259510,Shufflepuck Cantina Deluxe,Agharta Studio,NaN,"Dec 6, 2013",451,82,"100,000 .. 200,000",999,"{'Sports': 397, 'Indie': 60, 'VR': 52, 'Action...",...,False,False,False,False,False,True,False,False,False,False
1497,259530,Savant - Ascent,D-Pad Studio,Savant - Ascent,"Dec 4, 2013",2583,219,"200,000 .. 500,000",199,"{'Action': 125, 'Indie': 111, 'Great Soundtrac...",...,False,False,False,False,False,False,False,False,False,False
1498,259550,Hero of the Kingdom,Lonely Troops,Hero of the Kingdom,"Dec 20, 2012",4376,296,"200,000 .. 500,000",799,"{'Casual': 182, 'Adventure': 133, 'Indie': 108...",...,False,True,False,False,False,False,False,False,False,False


In [6]:
agg_reviews = game_reviews.groupby(by="app_id")['review_text'].apply(lambda x: ' '.join(x))

In [9]:
agg_reviews

app_id
10        Ruined my life. This will be more of a ''my ex...
20        When I got this for Christmas in 1999 along wi...
30        Even though its old, its better than Call of D...
40        Buy this game and join the community of one Th...
50        This is the first game created by Gearbox - th...
                                ...                        
562600    I must be the most ♥♥♥♥♥♥♥♥ person in the enti...
562700    its not fun and it takes multiple shots to tag...
563180    Surprisingly varied, if slightly too exacting ...
563400    Ludu is a pretty neat 'adventure' puzzle game....
563430    Do you like RPGs? Do you like exploring dungeo...
Name: review_text, Length: 9368, dtype: object